In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems

E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie

In [3]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )
@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end
function distance_homoclinic(p1, p2)
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    dist = sqrt( (x1-x2)^2 + (y1-y2)^2 + (z1-z2)^2 )
    return dist
end;

In [293]:
"""
arguments:
- model
- u0
- parameters
- times -- array type of [] which include time_integrate, time_transient
- integ_setting -- settings for method of integration
- t_int_ts -- time inerval for timeseries
- t_int_3d -- time inerval for 3d phase portreit
 - labels -- labels for abscissa and ordinate of timeseries
"""
function research_regime(model, u0, p, times, integ_setting;
        index_3d = [1, 2, 3], index_ts = 1,
        t_int_ts = nothing, t_int_3d = nothing,
        labels = ["time", ""], plot = true)
    
    ds = CoupledODEs(model, u0, p, diffeq = integ_setting)
    tr, trange = trajectory(ds, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    

    t_int_ts = check_t_int(t_int_ts, times)
    t_int_3d = check_t_int(t_int_3d, times)

    inter_3d = interactive_3d(tr, [3, 2, 1], t_int_3d)
    ts = timeseries([trange, tr], index_ts, t_int_ts, labels)
    
    if plot == true
    GLMakie.activate!()
    display(GLMakie.Screen(), inter_3d)
    display(GLMakie.Screen(), ts)
    end
    return [ds, tr, trange, inter_3d, ts]
end
function interactive_3d(data, index, interval)
    f = Figure(resolution = (800, 500))
    axis3 = Axis3(f[1, 1],
                xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
                xticklabelsize = 20, yticklabelsize = 20, zticklabelsize = 20,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)

    lines!(axis3,
        data[interval[1]:interval[2], index[1]], data[interval[1]:interval[2], index[2]],
        data[interval[1]:interval[2],index[3]], linewidth = 1.5, color = :deeppink)
    return f
end
function check_t_int(t_int, times)
    if t_int == nothing
        t_int = [1, times[1]]
    end
    return t_int
end
function timeseries(data, index, interval, labels)
    
    f = Figure(resolution = (900, 250))
    axis = Axis(f[1, 1], xlabel = labels[1], ylabel = labels[2],
            xlabelsize = 35, ylabelsize = 35,
            xticklabelsize = 28, yticklabelsize = 28,
            xgridvisible = false, ygridvisible = false)
    
    lines!(axis, data[1][interval[1]:interval[2]], data[2][interval[1]:interval[2], index],
            linewidth = 2.5, color = :deeppink)
    return f
end

timeseries (generic function with 1 method)

In [5]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [273]:
times = [time, tt]
time_int_ts = [1, 50000]
time_int_3d = [1, 100000]

2-element Vector{Int64}:
      1
 100000

In [325]:
U0 = 0.252; I0 = -1.72

time = 500; tt = 1000; tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]

u0 =  [12.769265363312517,0.645078134058814,0.46143383140811733]

3-element Vector{Float64}:
 12.769265363312517
  0.645078134058814
  0.46143383140811733

In [326]:
prev = u0

3-element Vector{Float64}:
 12.769265363312517
  0.645078134058814
  0.46143383140811733

In [327]:
output = research_regime(TM, u0, p,
    times, integ_set, t_int_ts = time_int_ts, t_int_3d = time_int_3d, plot = true)

5-element Vector{Any}:
 3-dimensional CoupledODEs
 3-dimensional StateSpaceSet{Float64} with 500001 points
 1000.0:0.001:1500.0
 Figure()
 Figure()

Multistability
U0 = 0.252; I0 = -1.72
[12.769265363312517,0.645078134058814,0.46143383140811733]

Double period
[5.703960241685617, 0.7514489808165569, 0.44878444809941526]

Homoclinic
[6.154581240346446, 0.7336262321641008, 0.4403107515803029]

In [328]:
lyapunovspectrum(output[1], time; Ttr = tt)

3-element Vector{Float64}:
 -9.655429969086706e-5
 -0.586573920678845
 -0.5778269077189608